<a href="https://colab.research.google.com/github/krkrkrrk/signate-technoprodesign/blob/main/(for__portfolio)technopro_design_compe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# テクノプロ・デザイン社 食品パッケージ画像解析チャレンジ（一般部門・学生部門）の解法

本コードは公開されていたベースラインを利用しており、自分で書いたコードは少ないです。

Warmup、FocalLoss、アンサンブルのためのコードは自分で追加しました。その他カスタマイズしている部分がいくつかあります。

また、最初は全くコードの意味が分かりませんでしたが、2か月間頑張って読み解き、今では大体のコードを理解できています。コメントはほとんど私が書いたものです。

## 初期設定

In [ ]:
!nvidia-smi

Mon Oct  9 08:43:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install colorama
!cp /content/drive/MyDrive/technoprodesign/data/train.zip .
!cp /content/drive/MyDrive/technoprodesign/data/test.zip .
!unzip train.zip
!unzip test.zip

In [ ]:
pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00


In [ ]:
pip install ttach

In [ ]:
!git clone https://github.com/davda54/sam.git

Cloning into 'sam'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 200 (delta 73), reused 64 (delta 58), pack-reused 104
Receiving objects: 100% (200/200), 659.65 KiB | 8.46 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [ ]:
pip install git+https://github.com/lessw2020/Ranger21.git

  Cloning https://github.com/lessw2020/Ranger21.git to /tmp/pip-req-build-s_r58cqr
  Running command git clone --filter=blob:none --quiet https://github.com/lessw2020/Ranger21.git /tmp/pip-req-build-s_r58cqr
  Resolved https://github.com/lessw2020/Ranger21.git to commit 1a96777278cdd14bc11afd865112724386d26a44
  Preparing metadata (setup.py) ... done
  Created wheel for ranger21: filename=ranger21-0.0.1-py3-none-any.whl size=24713 sha256=30865bf8e6020ba9aa2ff9351cb61337ac73b6c5944541d83061b1e7b55086e7
  Stored in directory: /tmp/pip-ephem-wheel-cache-z7owmj6b/wheels/ce/fa/a3/b9fb1d5b6129c81c852d06d951c3e9f5ce2b0dfa9ed4b5b455
Successfully built ranger21


## Import

In [ ]:
import os, shutil

#必要なライブラリのインポート
import re, gc, sys
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

import warnings, random
import cv2

warnings.filterwarnings("ignore")

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler
from sam.sam import SAM
from ranger21 import Ranger21

import torchvision
from torchvision import transforms
import torchvision.models as models
from torch.cuda.amp import GradScaler
import timm
from efficientnet_pytorch import EfficientNet

import yaml
from tqdm import tqdm
import time
import copy
from collections import defaultdict

from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

## パラメータの設定

In [ ]:
ARGS = {'DATA_DIR': '/content/drive/MyDrive/technoprodesign/data',
  'OUT_DIR': './',
  'model_name': 'eva02_large_patch14_448.mim_m38m_ft_in22k_in1k',
  'n_fold': 5,
  'epochs': 4,
  'criterion': 'CrossEntropy',
  'image_size': (448, 448),
  'train_batch_size': 16,
  'test_batch_size': 32,
  'seed': 777,
  'optimizer': 'AdamW',
  'learning_rate': 1.5e-05,
  'scheduler': 'warmup',
  'min_lr': 1e-06,
  'T_max': 500,
  'beta': (0.9, 0.999),
  'n_accumulate': 1,
  'clip_grad_norm': 'None',
  'apex': True,
  'num_classes': 2,
  'device': torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  }
ARGS

{'DATA_DIR': '/content/drive/MyDrive/technoprodesign/data',
 'OUT_DIR': './',
 'model_name': 'eva02_large_patch14_448.mim_m38m_ft_in22k_in1k',
 'n_fold': 5,
 'epochs': 4,
 'criterion': 'CrossEntropy',
 'image_size': (448, 448),
 'train_batch_size': 16,
 'test_batch_size': 32,
 'seed': 777,
 'optimizer': 'AdamW',
 'learning_rate': 1.5e-05,
 'scheduler': 'warmup',
 'min_lr': 1e-06,
 'T_max': 500,
 'beta': (0.9, 0.999),
 'n_accumulate': 1,
 'clip_grad_norm': 'None',
 'apex': True,
 'num_classes': 2,
 'device': device(type='cuda', index=0)}

In [ ]:
#Pythonのログを管理
#filenameはログが書き込まれるファイルの名前
def get_logger(filename):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    # __name__は現在のモジュール（スクリプト）の名前を示す特殊な変数。異なるモジュールごとに異なるロガーが生成される
    logger = getLogger(__name__)
    #ロガーのログレベルを設定。ここでは、ログレベルをINFOに設定。これにより、INFOレベル以上のログが出力される。
    logger.setLevel(INFO)
    # ログをファイルに出力するためのハンドラ（ハンドラはログ出力の仕組みを提供）を作成
    # filenameに指定された名前に.logを付け加えたファイル名で、ログが書き込まれるファイルを指定
    handler2 = FileHandler(filename=f"{filename}.log")
    # フォーマッタを設定。ログのメッセージ部分だけをフォーマットして出力するように指定
    handler2.setFormatter(Formatter("%(message)s"))
    # ロガーに先ほど作成したhandler2（ファイルへのログ出力ハンドラ）を追加。これにより、ログは指定されたファイルに出力される。
    logger.addHandler(handler2)
    return logger

#再現性を出すために必要な関数
def worker_init_fn(worker_id):
    torch.manual_seed(worker_id)
    random.seed(worker_id)
    np.random.seed(worker_id)
    torch.cuda.manual_seed(worker_id)
    os.environ['PYTHONHASHSEED'] = str(worker_id)

#再現性確保のため、ノートブック全体のシードを設定し、実行するたびに結果が同じになるようにする。
def set_seed(seed=777):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # CuDNNバックエンドで実行する場合、さらに2つのオプションを設定する必要がある
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # ハッシュシードに固定値を設定する
    os.environ['PYTHONHASHSEED'] = str(seed)

LOGGER = get_logger(ARGS['OUT_DIR']+'train')
set_seed(ARGS["seed"])

In [ ]:
# 交差検証
# data: データセット、num_splits: 分割するフォールドの数、seed: 乱数生成のシード
def create_folds(data, num_splits, seed):
    # 与えられたデータセットに新しい列 "kfold" を追加し、全ての要素を -1 で初期化
    data["kfold"] = -1

    #指定された数のフォールドにデータセットを分割（shuffle=True としているため、データはランダムにシャッフルされてから分割される）
    mskf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=seed)
    labels = ["label"]
    data_labels = data[labels].values

    # 分割されたデータセットのインデックスを使用して、各フォールドに属するデータの "kfold" 列の値を対応するフォールド番号に更新
    # t_ は訓練データのインデックス（のリストみたいなもの）、v_ は検証データのインデックス（のリストみたいなもの）
    # 「1つ目の分割されたデータ、2つ目の分割されたデータ、、、5つ目の分割されたデータ」って感じでforが回る
    for f, (t_, v_) in enumerate(mskf.split(data, data_labels)):
        # インデックスが v_ の行の "kfold" に、現在のフォールド番号 f を割り当てる（多分fは0～4）
        data.loc[v_, "kfold"] = f

    return data

train = pd.read_csv(f"{ARGS['DATA_DIR']}/train.csv")
train = create_folds(train, num_splits=ARGS["n_fold"], seed=ARGS["seed"])
print("Folds created successfully")

train.head()

Folds created successfully


,image_name,label,kfold
0,0000.png,0,2
1,0001.png,1,4
2,0002.png,1,2
3,0003.png,1,2
4,0004.png,0,0


In [ ]:
train["label"].value_counts()

1    1182
0     994
Name: label, dtype: int64

In [ ]:
train['kfold'].value_counts()

0    436
2    435
4    435
3    435
1    435
Name: kfold, dtype: int64

## 画像の読み込みと前処理

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, transform, data_type):
        self.df = df
        self.data_type = data_type
        self.images = []

        if self.data_type == "train":
            self.image_paths = df['image_name']
            self.labels = df['label']
        if self.data_type == "test":
            self.image_paths = df[0]

        self.transform = transform

        # あらかじめRAMに画像をロードしておく
        for image_path in self.image_paths:
            if self.data_type == "train":
                image = cv2.imread(f"/content/train/{image_path}")
            if self.data_type == "test":
                image = cv2.imread(f"/content/test/{image_path}")

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            self.images.append(image)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index: int):
        image = self.images[index]

        image = self.transform(image=image)["image"]

        if self.data_type == "train":
            label = self.labels[index]
            label = torch.tensor(label, dtype=torch.long)
            return image, label

        if self.data_type == "test":
            return image

In [ ]:
import albumentations as albu
from albumentations.pytorch import transforms as AT
import torchvision.transforms as T

# Augumentation用
#transforms.Composeを使う方法もあるが、albumentatioの方がより高速かつ柔軟らしい
image_transform = albu.Compose([
    albu.Resize(ARGS["image_size"][0], ARGS["image_size"][1]),
    albu.HorizontalFlip(p=0.5),
    albu.VerticalFlip(p=0.5),
    albu.RandomRotate90(),
    albu.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    AT.ToTensorV2()
])

## 学習用関数定義
`torch.cuda.amp.autocast(enabled=ARGS["apex"]):`を入れると計算の高速化とメモリの節約ができます。

In [ ]:
def train_one_epoch(model, optimizer, scheduler, train_loader, device, epoch):
    model.train()
    dataset_size = 0
    running_loss = 0.0
    running_score = []
    running_score_y = []

    # GradScalerとは「勾配をスケール（大きくする）するもの」
    # float16で表現できる桁数は限られており，小さい数値はアンダーフローで消えてしまう
    # 特に勾配計算ではアンダーフローが起きやすく、それを防ぐために勾配計算の前に勾配を大きくする必要がある
    # apexとは「NVIDIAのAMP機能」
    # AMP機能とは「通常float32で計算するものをfloat16で計算して高速化するための技術」
    # AMP = Automatic Mixed Precision = 自動混合精度

    # つまり、このコードは「勾配のスケールをfloat16でやりますよ」ってこと
    scaler = GradScaler(enabled=ARGS["apex"])

    train_loss = []
    # ループの進行状況を表示するためのやつ
    bar = tqdm(enumerate(train_loader), total=len(train_loader))
    # トレーニングデータローダーからバッチごとの画像とターゲットを取得。bar（進捗バー）によってループの進行状況を表示
    for step, (images, targets) in bar:
      # バッチ内の画像データと正解ラベルを計算デバイス（GPUまたはCPU）に移動
      images = images.to(device)
      targets = targets.to(device)

      batch_size = targets.size(0)
      # AMPを使用して、計算をfloat16で行う。これにより、計算の速度が向上し、メモリ使用量が減少する
      with torch.cuda.amp.autocast(enabled=ARGS["apex"]):
          # モデルに入力データを与えて、出力を計算
          outputs = model(images)
          # 計算された出力とターゲットとの間で損失を計算
          loss = criterion(ARGS, outputs, targets)

      # 損失をscalerでかなり大きくしてから逆伝播
      scaler.scale(loss).backward()

      # 勾配の大きさを閾値内に制限することで、勾配爆発を防ぐ
      # 勾配爆発とは「ニューラルネットワークのトレーニング中に勾配が非常に大きな値を取ることで発生する問題」
      # 勾配が非常に大きな値を持つと、重みの更新量も大きくなり、モデルが不安定になったり発散したりする可能性がある
      if ARGS["clip_grad_norm"] != "None":
          grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), ARGS["clip_grad_norm"])
      else:
          grad_norm = None

      # 逆伝播が終わってからもとの勾配の大きさに直してパラメタを更新
      scaler.step(optimizer)
      scaler.update()
      # モデル内のすべての勾配をゼロにリセット。これにより、次のバッチの勾配計算が独立して行われる
      optimizer.zero_grad()

      # 学習率スケジューラが存在する場合、学習率を更新
      if scheduler is not None:
          scheduler.step()

      # このバッチの訓練損失を train_loss リストに追加
      # item()は「PyTorchのテンソル（Tensor）からPythonの数値（スカラー）を取得するためのメソッド」
      train_loss.append(loss.item())

      # トレーニングデータのバッチごとの損失を累積
      # 現在のバッチの損失（loss）をバッチサイズ（batch_size）と掛けて加算。これにより、トレーニングデータ全体に対する累積損失が計算される
      running_loss += (loss.item() * batch_size)
      # データセットのサンプル数をバッチサイズごとに累積。これにより、トレーニングデータ全体のサンプル数が計算される
      dataset_size += batch_size

      # エポックごとの平均損失を計算
      epoch_loss = running_loss / dataset_size

      # モデルの出力と正解ラベルをリストに追加
      # detach().cpu().numpy()は「PyTorchテンソルをNumpy配列に変換するためのもの」
      # detach()は「outputsを元のテンソルの計算グラフから切り離し、新しいテンソルとして生成するもの」
      # cpu()は「テンソルをCPUメモリに移動させるもの」
      # numpy()は「テンソルをNumpy配列に変換するもの」
      running_score.append(outputs.detach().cpu().numpy())
      running_score_y.append(targets.detach().cpu().numpy())

      # スコアを計算。get_score 関数は別の場所で定義されており、戻り値は(accuracy, auc_score)
      score = get_score(running_score_y, running_score)

      # 進捗バーにエポック、損失、スコアなどの情報を表示
      bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                      Train_Acc=score[0],
                      Train_Auc=score[1],
                      LR=optimizer.param_groups[0]['lr']
                          )
    #  ガベージコレクションを実行してメモリを解放
    gc.collect()
    # score = (accuracy, auc_score)
    return epoch_loss, score

In [ ]:
# 検証では逆伝搬を行わないため、勾配計算を行わない
# この@から始まるやつは「デコレータ」っていうらしい。関数の処理の前後にコメントなどの独自の処理を加えたりできるらしい
@torch.no_grad()
def valid_one_epoch(args, model, optimizer, valid_loader, epoch):
    # モデルを評価モードに設定
    # これにより、モデル内部の学習可能なパラメータが凍結され、バッチ正規化やドロップアウトのような訓練中にのみ適用される機能が無効になる
    model.eval()

    dataset_size = 0
    running_loss = 0.0
    preds = []
    valid_targets = []
    softmax = nn.Softmax()

    bar = tqdm(enumerate(valid_loader), total=len(valid_loader))
    # このforはバッチ単位で処理される
    for step, (images, targets) in enumerate(valid_loader):
      images = images.to(args["device"])
      targets = targets.to(args["device"])
      batch_size = targets.size(0)
      # 誤差逆伝搬を無効にして、計算中に勾配の計算を行わないようにする
      with torch.no_grad():
        outputs = model(images)
        # モデルの出力をソフトマックス関数で確率分布に変換
        predict = outputs.softmax(dim=1)
        loss = criterion(args, outputs, targets)

      running_loss += (loss.item() * batch_size)
      dataset_size += batch_size

      epoch_loss = running_loss / dataset_size

      preds.append(predict.detach().cpu().numpy())
      valid_targets.append(targets.detach().cpu().numpy())

      # 正解ラベルがすべて同じ場合は、評価をスキップ。これにより、計算が不要な場合の効率化を図る
      if len(set(np.concatenate(valid_targets))) == 1:
          continue
      score = get_score(valid_targets, preds)

      bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                      Valid_Acc=score[0],
                      Valid_Auc=score[1],
                      LR=optimizer.param_groups[0]['lr'])

    # predsは0～1の確率
    return epoch_loss, preds, valid_targets, score


In [ ]:
# K分割交差検証の1つの分割（fold）におけるモデルのトレーニングと評価を行う関数
def one_fold(model, optimizer, scheduler, train_loader, valid_loader, device, num_epochs, fold):

    # GPUが利用可能な場合に、利用されるGPUの名前を表示
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))

    # 処理の開始時刻を記録
    start = time.time()
    # ベストスコアを達成したエポックでのモデルの重みを保持する変数。これはただの初期化
    best_model_wts = copy.deepcopy(model.state_dict())
    # ベストエポックスコアを無限大に初期化
    best_epoch_score = np.inf
    # 単純な初期化
    best_prediction = None

    # ベストスコアをマイナス無限大に初期化
    best_score = -np.inf
    train_epoch_loss_list = []
    valid_epoch_loss_list = []
    # train_one_epoch関数をepochの回数分実行するって仕組み
    for epoch in range(1, 1+num_epochs):
      # train_one_epochは別のセルに定義されている
      # train_score = (accuracy, auc_score)
      train_epoch_loss, train_score = train_one_epoch(model,
                                                      optimizer,
                                                      scheduler,
                                                      train_loader=train_loader,
                                                      device=device,
                                                      epoch=epoch)

      train_acc, train_auc = train_score

      # loss、予測確率、正解ラベル、検証データでのスコア (accuracy, auc_score)
      valid_epoch_loss, predictions, valid_targets, valid_score = valid_one_epoch(ARGS,
                                                                                  model,
                                                                                  optimizer,
                                                                                  valid_loader,
                                                                                  epoch=epoch)
      valid_acc, valid_auc = valid_score
      train_epoch_loss_list.append(train_epoch_loss)
      valid_epoch_loss_list.append(valid_epoch_loss)

      # Epochが終わるごとにlossやスコアを表示
      LOGGER.info(f'Epoch {epoch} - avg_train_loss: {train_epoch_loss:.4f}  avg_valid_loss: {valid_epoch_loss:.4f}')
      LOGGER.info(f'Epoch {epoch} - Train Acc: {train_acc:.4f}  Train Auc: {train_auc:.4f}  Valid Acc: {valid_acc:.4f}  Valid Auc: {valid_auc:.4f}')

      # あるEpochのaucが今までのaucを超えていたらベストaucを更新してモデルの重みを保存
      #if valid_epoch_loss <= best_score:
       # best_score = valid_epoch_loss
      if valid_auc >= best_score:
        best_score = valid_auc

        print(f"{b_}Validation Score Improved ({best_epoch_score} ---> {valid_auc})")
        best_epoch_score = valid_auc
        # モデルの重みを保存
        best_model_wts = copy.deepcopy(model.state_dict())
        # PATH = f"Score-Fold-{fold}.bin"
        # 次以降のEpochでベストaucが更新されたら、binファイルを上書き保存
        PATH = ARGS["OUT_DIR"] + f"Score-Fold-{fold}.bin"
        torch.save(model.state_dict(), PATH)
        # Save a model file from the current directory
        print(f"Model Saved{sr_}")

        # predictionsは[[[0.8, 0.2], [0.1, 0.9]], [[0.4, 0.6], [0.7, 0.3]] , [[0.2, 0.8], [0.8, 0.2]]]みたいな３次元配列
        # concatenateすることで、[[0.8, 0.2], [0.1, 0.9], [0.4, 0.6], [0.7, 0.3] , [0.2, 0.8], [0.8, 0.2]]というような２次元配列になる
        # その配列の１列目がそのまま予測確率になってるってわけ
        best_prediction = np.concatenate(predictions, axis=0)[:,1]

    end = time.time()
    time_elapsed = end - start

    # 1fold終了後にトレーニングにかかった時間とベストスコアを表示
    LOGGER.info('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    LOGGER.info("Best Score: {:.4f}".format(best_epoch_score))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_prediction, valid_targets, best_epoch_score, train_epoch_loss_list, valid_epoch_loss_list

## モデルや損失関数、最適化関数等の定義

In [ ]:
def create_model(args):
    # timmを使用して事前学習済みモデルを読み込む
    model = timm.create_model(args["model_name"], pretrained=True)
    model.reset_classifier(args["num_classes"])
    return model

#https://take-tech-engineer.com/pytorch-focal-loss/
#https://deecode.net/?p=925
class Focal_MultiLabel_Loss(nn.Module):
    def __init__(self, gamma):
      #親クラス（nn.Module）のコンストラクタを呼び出し
      super(Focal_MultiLabel_Loss, self).__init__()
      self.gamma = gamma
      #損失関数内でシグモイド計算がされているので、train_one_epochの最後にシグモイド関数を適用する必要がない
      self.bceloss = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, outputs, labels):
      bce = self.bceloss(outputs, labels)
      #torch.exp()は、入力テンソルの要素の指数を取得して、新しいテンソルを返す
      bce_exp = torch.exp(-bce)
      #二値交差エントロピー損失の指数を計算
      focal_loss = (1-bce_exp)**self.gamma * bce
      return focal_loss.mean()

# class_weightsは「クラスごとの重み」。デフォルトではNone、つまり全クラス同じ重み
def criterion(args, outputs, labels, class_weights=None):
    if args['criterion'] == 'CrossEntropy':
      return nn.CrossEntropyLoss(weight=class_weights).to(args["device"])(outputs, labels)
    elif args['criterion'] == "FocalLoss":
      #outputsは２次元テンソルだが、labelsは１次元テンソル。なので、outputs[:, 1]で2列目だけを取り出す
      #outputsはfloat32で、labelsはint64であるため、labelsをfloatに変換
      return Focal_MultiLabel_Loss(gamma=2).to(args["device"])(outputs[:, 1], labels.float())
    elif args['criterion'] == "None":
        return None

def fetch_optimizer(optimizer_parameters, args, lr, betas, train_loader, optimizer_name):
    if optimizer_name == "Adam":
        optimizer = optim.Adam(optimizer_parameters, lr=lr, betas=betas)
    elif optimizer_name == "AdamW":
        optimizer = optim.AdamW(optimizer_parameters, lr=lr, betas=betas)
    elif optimizer_name == "RAdam":
        optimizer = optim.RAdam(optimizer_parameters, lr=lr, betas=betas)
    elif optimizer_name == "Lion":
        optimizer = Lion(optimizer_parameters, weight_decay=0.1, betas=(0.95,0.98), lr=lr)
    elif optimizer_name == "SAM":
        base_optimizer = optim.SGD
        optimizer = SAM(optimizer_parameters, base_optimizer, lr = lr, momentum=0.9)
    elif optimizer_name == "Ranger21":
        optimizer = Ranger21(optimizer_parameters, lr = lr, num_epochs = args['epochs'], num_batches_per_epoch = len(train_loader))
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(optimizer_parameters, lr = lr)
    elif optimizer_name == "momentum_SGD":
        optimizer = optim.SGD(optimizer_parameters,lr=lr,momentum=0.9)
    elif optimizer_name == "Adagrad":
        optimizer = optim.Adagrad(optimizer_parameters, lr = lr)
    elif optimizer_name == "RMSprop":
        optimizer = optim.RMSprop(optimizer_parameters, lr = lr)
    elif optimizer_name == "Adadelta":
        optimizer = optim.Adadelta(optimizer_parameters, lr = lr)
    return optimizer

def lr_lambda(current_step):
    warmup_steps = 108
    #1epochごとのステップ数は108～109で、4epochあるので、total_step=432～436
    #len(train_loader)は1epochごとのステップ数
    total_steps = 436
    if current_step < warmup_steps:
        return float(current_step) / float(max(1, warmup_steps))
    else:
        #progressの範囲は[0, 1]
        progress = (current_step - warmup_steps) / float(max(1, total_steps - warmup_steps))
        #cosine関数の結果は[-1, 1]の範囲になり、1.0を足すため、[0, 2]の範囲になってしまう
        #そのため、0.5を掛けることで、[0, 1]の範囲になるようにしている
        return 0.5 * (1.0 + np.cos(np.pi * progress))

def fetch_scheduler(args, optimizer):
    # 学習率を余弦関数の形状で変化させるもので、トレーニングの進行に従って学習率を調整
    # T_maxはエポック数の最大値、eta_minは最小学習率
    if args['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=args['T_max'],
                                                   eta_min=args['min_lr'])

    # 一定の周期で学習率をリセットすることで、訓練中の学習率の変化を制御
    # T_0は最初のリスタートまでのエポック数
    elif args['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=args['T_0'],
                                                             eta_min=args['min_lr'])

    # 参考：https://tam5917.hatenablog.com/entry/2022/09/16/214700
    elif args['scheduler'] == 'warmup':
        scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda)

    elif args['scheduler'] == "None":
        scheduler = None

    return scheduler

def get_score(y_trues, y_preds):

    predict_list, targets_list = np.concatenate(y_preds, axis=0), np.concatenate(y_trues)
    predict_list_proba = predict_list.copy()[:, 1]
    predict_list = predict_list.argmax(axis=1)

    accuracy = accuracy_score(predict_list, targets_list)
    auc_score = roc_auc_score(targets_list, predict_list_proba)

    return (accuracy, auc_score)

def prepare_loaders(args, train, image_transform, fold):
    #特定のクロスバリデーションフォールドを除外したデータセットを作成。
    #reset_indexにより旧インデックスをリセットし、新しいインデックスを付け直す
    #drop=True は、旧インデックスを列として追加しないように指定するためのもの
    df_train = train[train.kfold != fold].reset_index(drop=True)
    df_valid = train[train.kfold == fold].reset_index(drop=True)

    train_dataset = CustomDataset(df_train, image_transform, data_type="train")
    valid_dataset = CustomDataset(df_valid, image_transform, data_type="train")

    # num_workers=2などに設定することで、multi-process data loadingとなり、処理が高速化される
    # pin_memory=Trueにすることで、automatic memory pinningが使用できる。CPUのメモリ領域がページングされないようになり、高速化が期待される
    train_loader = DataLoader(train_dataset, batch_size=args['train_batch_size'],
                              worker_init_fn=worker_init_fn(args["seed"]),
                              num_workers=os.cpu_count(),
                              shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=args['test_batch_size'],
                              num_workers=os.cpu_count(),
                              shuffle=False, pin_memory=True)

    return train_loader, valid_loader

## メイン（アンサンブル）

乱数のみを変えたモデルを5つ用意する。1つずつ交差検証による学習→推論を行い、それぞれの結果を最終的に平均して結果を出力


In [ ]:
@torch.no_grad()
def test_fn(model, dataloader, device):
    model.eval()

    dataset_size = 0
    running_loss = 0.0
    predict_list = []

    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, images in bar:
        images = images.to(device)
        with torch.no_grad():
            outputs = model(images)
            #出力にソフトマックス関数を適用
            predicts = outputs.softmax(dim=1)

        predicts = predicts.cpu().detach().numpy()
        predict_list.append(predicts)
    predict_list = np.concatenate(predict_list, axis=0)
    #予測値が1である確率を提出します。
    predict_list = predict_list[:, 1]
    gc.collect()

    return predict_list

def one_model_inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = create_model(ARGS)
        model = model.to(device)

        #学習済みモデルの読み込み
        model.load_state_dict(torch.load(path))
        model.eval()

        print(f"Getting predictions for model {i+1}")
        preds = test_fn(model, dataloader, device)
        final_preds.append(preds)

    #final_predsは元々2次元配列で、np.arrayによりnumpy配列に変換
    final_preds = np.array(final_preds)
    #numpy配列においてはmeanメソッド(axis=0)により列の平均値を取ることができる。
    #例えば、[[1,2],[3,4]]のとき、[2,3]になる
    #つまり、1個1個のモデルの予測結果の平均値を取ったものを最終的な予測結果にしている
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

train_copy = train.copy()
train_epoch_loss_list_list = []
valid_epoch_loss_list_list = []

LOGGER.info(ARGS)

models = []
final_preds = []
for i in range(5):
    ARGS["seed"] = i+1
    print(f"{y_}====== Model: {i+1} ======")
    LOGGER.info(f"========== Model: {i+1} training ==========")

    for fold in range(0, ARGS["n_fold"]):
        print(f"{y_}====== Fold: {fold} ======{sr_}")
        LOGGER.info(f"========== fold: {fold} training ==========")

        # データローダーの作成
        train_loader, valid_loader = prepare_loaders(args=ARGS, train=train, image_transform=image_transform, fold=fold)

        #フォールドごとにモデルを初期化
        model = create_model(ARGS)
        model = model.to(ARGS["device"])

        #損失関数・最適化関数の定義
        optimizer = fetch_optimizer(model.parameters(), args=ARGS, optimizer_name=ARGS["optimizer"], lr=ARGS["learning_rate"], betas=ARGS["beta"], train_loader=train_loader)
        scheduler = fetch_scheduler(args=ARGS, optimizer=optimizer)

        model, predictions, targets, scores, train_epoch_loss_list, valid_epoch_loss_list = one_fold(model,
                                                                                                     optimizer,
                                                                                                     scheduler,
                                                                                                     train_loader,
                                                                                                     valid_loader,
                                                                                                     device=ARGS["device"],
                                                                                                     num_epochs=ARGS["epochs"],
                                                                                                     fold=fold)

        # OOFとはOut Of Fold (Prediction)から来ており、文字通りフォールドから出てきた予測値という意味
        train_copy.loc[train_copy[train_copy.kfold == fold].index, "oof"] = predictions

    scores = roc_auc_score(train_copy["label"].values, train_copy["oof"].values)
    LOGGER.info(f"========== CV ==========")
    LOGGER.info(f"CV: {scores:.4f}")



    #################　ここから評価　##################
    LOGGER.info(f"========== Model: {i} evaluation ==========")
    model.eval()

    #sample_submit.csvを読み込む
    submit = pd.read_csv(f"{ARGS['DATA_DIR']}/sample_submit.csv", header=None)
    submit.head()

    # test用のデータ拡張
    image_transform_test = albu.Compose([
        albu.Resize(ARGS["image_size"][0], ARGS["image_size"][1]),
        albu.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        AT.ToTensorV2 ()
        ])
    test_dataset = CustomDataset(submit, image_transform_test, data_type="test")
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

    MODEL_PATHS = [
        f"{ARGS['OUT_DIR']}/Score-Fold-{i}.bin" for i in range(ARGS["n_fold"])
    ]

    preds = one_model_inference(MODEL_PATHS, test_loader, ARGS["device"])
    final_preds.append(preds)

    #ガベージコレクション
    del model, train_loader, valid_loader, test_loader
    _ = gc.collect()
    torch.cuda.empty_cache()
    print()

INFO:__main__:{'DATA_DIR': '/content/drive/MyDrive/technoprodesign/data', 'OUT_DIR': './', 'model_name': 'eva02_large_patch14_448.mim_m38m_ft_in22k_in1k', 'n_fold': 5, 'epochs': 4, 'criterion': 'CrossEntropy', 'image_size': (448, 448), 'train_batch_size': 16, 'test_batch_size': 32, 'seed': 777, 'optimizer': 'AdamW', 'learning_rate': 1.5e-05, 'scheduler': 'warmup', 'min_lr': 1e-06, 'T_max': 500, 'beta': (0.9, 0.999), 'n_accumulate': 1, 'clip_grad_norm': 'None', 'apex': True, 'num_classes': 2, 'device': device(type='cuda', index=0)}
INFO:__main__:========== Model: 1 training ==========
INFO:__main__:========== fold: 0 training ==========


====== Model: 1 ======
====== Fold: 0 ======


---modelのconfig---
{'url': '', 'hf_hub_id': 'timm/eva02_large_patch14_448.mim_m38m_ft_in22k_in1k', 'architecture': 'eva02_large_patch14_448', 'tag': 'mim_m38m_ft_in22k_in1k', 'custom_load': False, 'input_size': (3, 448, 448), 'fixed_input_size': True, 'interpolation': 'bicubic', 'crop_pct': 1.0, 'crop_mode': 'squash', 'mean': (0.48145466, 0.4578275, 0.40821073), 'std': (0.26862954, 0.26130258, 0.27577711), 'num_classes': 1000, 'pool_size': None, 'first_conv': 'patch_embed.proj', 'classifier': 'head', 'license': 'mit'}
[INFO] Using GPU: NVIDIA A100-SXM4-40GB



  0%|          | 0/14 [00:21<?, ?it/s, Epoch=1, LR=1.5e-5, Valid_Acc=0.826, Valid_Auc=0.901, Valid_Loss=0.408]
INFO:__main__:Epoch 1 - avg_train_loss: 0.4868  avg_valid_loss: 0.4082
INFO:__main__:Epoch 1 - Train Acc: 0.7731  Train Auc: 0.8151  Valid Acc: 0.8257  Valid Auc: 0.9011


Validation Score Improved (inf ---> 0.9010877170663446)
Model Saved


...

100%|██████████| 69/69 [01:36<00:00,  1.40s/it]


---modelのconfig---
{'url': '', 'hf_hub_id': 'timm/eva02_large_patch14_448.mim_m38m_ft_in22k_in1k', 'architecture': 'eva02_large_patch14_448', 'tag': 'mim_m38m_ft_in22k_in1k', 'custom_load': False, 'input_size': (3, 448, 448), 'fixed_input_size': True, 'interpolation': 'bicubic', 'crop_pct': 1.0, 'crop_mode': 'squash', 'mean': (0.48145466, 0.4578275, 0.40821073), 'std': (0.26862954, 0.26130258, 0.27577711), 'num_classes': 1000, 'pool_size': None, 'first_conv': 'patch_embed.proj', 'classifier': 'head', 'license': 'mit'}
Getting predictions for model 4


100%|██████████| 69/69 [01:36<00:00,  1.40s/it]


---modelのconfig---
{'url': '', 'hf_hub_id': 'timm/eva02_large_patch14_448.mim_m38m_ft_in22k_in1k', 'architecture': 'eva02_large_patch14_448', 'tag': 'mim_m38m_ft_in22k_in1k', 'custom_load': False, 'input_size': (3, 448, 448), 'fixed_input_size': True, 'interpolation': 'bicubic', 'crop_pct': 1.0, 'crop_mode': 'squash', 'mean': (0.48145466, 0.4578275, 0.40821073), 'std': (0.26862954, 0.26130258, 0.27577711), 'num_classes': 1000, 'pool_size': None, 'first_conv': 'patch_embed.proj', 'classifier': 'head', 'license': 'mit'}
Getting predictions for model 5


100%|██████████| 69/69 [01:36<00:00,  1.40s/it]


## 結果の表示

In [ ]:
def ensemble_inference(final_preds, device):
    #final_predsは元々2次元配列で、np.arrayによりnumpy配列に変換
    final_preds = np.array(final_preds)

    #相関係数を表示。相関がありすぎるとアンサンブルの意味がない（https://qiita.com/koshian2/items/d569cd71b0e082111962）
    #.Tは転置
    df = pd.DataFrame(final_preds).T
    print(abs(df.corr()))
    df.to_csv(f"{ARGS['OUT_DIR']}/final_preds.csv", index=False, header=None)

    #numpy配列においてはmeanメソッド(axis=0)により列の平均値を取ることができる。
    #例えば、[[1,2],[3,4]]のとき、[2,3]になる
    #つまり、1個1個のモデルの予測結果の平均値を取ったものを最終的な予測結果にしている
    final_preds_mean = np.mean(final_preds, axis=0)
    return final_preds_mean

In [ ]:
predict_list = ensemble_inference(final_preds, ARGS["device"])
# predict_listの中身は何だ？（最終的な予測結果を格納した1次元配列だよ）
#submitは「画像名」と「確率」の2列で構成されている。確率の部分だけを変えるために[1]を指定
#（pandasの配列だからこの指定の仕方でうまくいく。普通の配列だと列を抜き出すにはforを使う必要がある）
submit[1] = predict_list
submit.head()

          0         1         2         3         4
0  1.000000  0.944802  0.944188  0.950513  0.947504
1  0.944802  1.000000  0.941866  0.944649  0.945180
2  0.944188  0.941866  1.000000  0.947032  0.947222
3  0.950513  0.944649  0.947032  1.000000  0.948143
4  0.947504  0.945180  0.947222  0.948143  1.000000


,0,1
0,2176.png,0.069441
1,2177.png,0.533749
2,2178.png,0.035690
3,2179.png,0.998782
4,2180.png,0.016801


In [ ]:
submit.to_csv(f"{ARGS['OUT_DIR']}/submission.csv", index=False, header=None)